# The Tensor objects

In [1]:
import pixelprism.math as pm
import numpy as np

## Creation

A `Tensor` is PixelPrism's friendly wrapper around NumPy arrays. It keeps the data, the shape, and the dtype in one tidy place, while staying easy to construct in a bunch of ways. Let's tour every creation path in `pixelprism/math/tensor.py`.

### 1) Direct constructor: scalars, vectors, matrices

The `Tensor` constructor accepts numbers, lists, nested lists, and NumPy arrays. Shapes are inferred automatically.

In [2]:
scalar = pm.Tensor(3)
scalar


tensor(3, dtype=Integers, shape=(), mutable=True)

In [3]:
vector = pm.Tensor([1, 2, 3])
matrix = pm.Tensor([[1, 2], [3, 4]])
vector, matrix


(tensor([1, 2, 3], dtype=Reals, shape=(3,), mutable=True),
 tensor([[1, 2], [3, 4]], dtype=Reals, shape=(2, 2), mutable=True))

A single number creates a rank-0 tensor (a scalar). A 1D list becomes a vector, and nested lists become matrices or higher-rank tensors.

### 2) From NumPy arrays

If you already have NumPy data, pass it directly. NumPy scalars also work.

In [4]:
np_vec = np.array([10, 20, 30], dtype=np.int32)
np_scalar = np.array(2.5)

from_np_vec = pm.Tensor(np_vec)
from_np_scalar = pm.Tensor(np_scalar)
from_np_vec, from_np_scalar


(tensor([10, 20, 30], dtype=Integers, shape=(3,), mutable=True),
 tensor(2.5, dtype=Reals, shape=(), mutable=True))

### 3) Static helpers for clarity

These are thin wrappers around the constructor, but they are explicit and readable in tutorials or pipelines.

In [5]:
explicit_np = pm.Tensor.from_numpy(np.arange(6).reshape(2, 3))
print(explicit_np)


tensor([[0, 1, 2], [3, 4, 5]], dtype=Integers, shape=(2, 3), mutable=True)


In [6]:
explicit_list = pm.Tensor.from_list([1, 2, 3], dtype=pm.Z)
explicit_list


tensor([1, 2, 3], dtype=Integers, shape=(3,), mutable=True)

In [ ]:
%%sql


`from_list` requires a `DType` so there is no ambiguity about integers vs floats.

### 4) Zeros factory

`Tensor.zeros` creates a tensor filled with zeros. The shape can be an int, a tuple, a list, or a `TensorShape`.

In [7]:
zeros_tuple = pm.t_zeros((2, 3))
zeros_int = pm.t_zeros(4)
zeros_shape = pm.t_zeros(pm.ts_matrix(2, 2))
zeros_tuple, zeros_int, zeros_shape


(tensor([[0, 0, 0], [0, 0, 0]], dtype=Integers, shape=(2, 3), mutable=True),
 tensor([0, 0, 0, 0], dtype=Integers, shape=(4,), mutable=True),
 tensor([[0, 0], [0, 0]], dtype=Integers, shape=(2, 2), mutable=True))

### 5) Dtype and mutability knobs

The constructor accepts `dtype` and `mutable`. `dtype` can be a `pm.DType`, a NumPy dtype, or a Python type (`int`, `float`, `bool`, `complex`). `mutable=False` makes the tensor read-only for in-place ops.

In [8]:
float_tensor = pm.Tensor([1, 2, 3], dtype=pm.DType.R)
bool_tensor = pm.Tensor([1, 0, 1], dtype=bool)
immutable = pm.Tensor([1, 2, 3], mutable=False)
float_tensor, bool_tensor, immutable


(tensor([1.0, 2.0, 3.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([True, False, True], dtype=Boolean, shape=(3,), mutable=True),
 tensor([1, 2, 3], dtype=Reals, shape=(3,), mutable=False))

## Algebra

Tensors are made for algebra. Every operation below is **elementwise**: each number talks only to its matching partner. This mirrors how we build models in machine learning: data flows through layers as a chain of simple, reliable transforms.

### 1) The four basics: +, -, *, /

Think of these as the arithmetic gears of a neural network: add a bias, scale activations, normalize, or compute residuals.

In [9]:
x = pm.Tensor([1.0, 2.0, 3.0])
y = pm.Tensor([10.0, 20.0, 30.0])

x_plus_y = x + y
x_minus_y = x - y
x_times_y = x * y
x_div_y = x / y

x_plus_y, x_minus_y, x_times_y, x_div_y


(tensor([11.0, 22.0, 33.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([-9.0, -18.0, -27.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([10.0, 40.0, 90.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([0.1, 0.1, 0.1], dtype=Reals, shape=(3,), mutable=True))

You can also mix tensors with scalars. Broadcasting applies the scalar across every element.

In [10]:
shift = x + 0.5
scale = x * 2
normalize = x / 10
shift, scale, normalize


(tensor([1.5, 2.5, 3.5], dtype=Reals, shape=(3,), mutable=True),
 tensor([2.0, 4.0, 6.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([0.1, 0.2, 0.3], dtype=Reals, shape=(3,), mutable=True))

You can also mix tensors with Numpy array. Broadcasting applies the scalar across every element.

In [11]:
shift = x + np.array(0.5)
shift_vector = x + np.array([0.5, 0.5, 0.5])
scale = x * np.array(2)
normalize = x / np.array([10, 10, 10])
shift, scale, normalize

(tensor([1.5, 2.5, 3.5], dtype=Reals, shape=(3,), mutable=True),
 tensor([2.0, 4.0, 6.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([0.1, 0.2, 0.3], dtype=Reals, shape=(3,), mutable=True))

### 2) Power moves: `**`, `pow`, and friends

In ML, exponents show up in loss functions (squared error), normalization, and feature engineering.

In [12]:
squared_op = x ** 2
squared_method = x.pow(2)

cube = x ** 3
fractional = x ** 0.5

squared_op, squared_method, cube, fractional


(tensor([1.0, 4.0, 9.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([1.0, 4.0, 9.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([1.0, 8.0, 27.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([1.0, 1.4142135623730951, 1.7320508075688772], dtype=Reals, shape=(3,), mutable=True))

`**` and `pow` are equivalent; choose the style that reads best in your notebook.

### 3) Convenience helpers: `square()` and `cbrt()`

These are readable shortcuts for common transforms. `square()` is the workhorse of least-squares losses, while `cbrt()` is a gentler nonlinearity.

In [13]:
square = x.square()
root3 = x.cbrt()

square, root3


(tensor([1.0, 4.0, 9.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([1.0, 1.2599210498948734, 1.4422495703074083], dtype=Reals, shape=(3,), mutable=True))

### 4) Advanced unary operations

These functions show up everywhere in mathematics and machine learning: exponentials and logs drive probability and optimization, while absolute values underpin L1 regularization and robust losses.

We keep inputs in safe domains (e.g., positive values for logs) to avoid undefined results.

In [14]:
u = pm.Tensor([0.1, 0.5, 1.0])

recip = u.reciprocal()  # 1/x
exp_u = u.exp()
exp2_u = u.exp2()
expm1_u = u.expm1()

recip, exp_u, exp2_u, expm1_u


(tensor([10.0, 2.0, 1.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([1.1051709180756477, 1.6487212707001282, 2.718281828459045], dtype=Reals, shape=(3,), mutable=True),
 tensor([1.0717734625362931, 1.4142135623730951, 2.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([0.10517091807564763, 0.6487212707001282, 1.718281828459045], dtype=Reals, shape=(3,), mutable=True))

In [15]:
v = pm.Tensor([1.0, 2.0, 10.0])

log_v = v.log()
log2_v = v.log2()
log10_v = v.log10()
log1p_v = v.log1p()

log_v, log2_v, log10_v, log1p_v


(tensor([0.0, 0.6931471805599453, 2.302585092994046], dtype=Reals, shape=(3,), mutable=True),
 tensor([0.0, 1.0, 3.321928094887362], dtype=Reals, shape=(3,), mutable=True),
 tensor([0.0, 0.3010299956639812, 1.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([0.6931471805599453, 1.0986122886681096, 2.3978952727983707], dtype=Reals, shape=(3,), mutable=True))

`log1p` and `expm1` are numerically stable for small values and often used in loss functions and probability computations.

In [16]:
w = pm.Tensor([-3.0, -1.5, 2.0])

abs_w = w.absolute()
abs_alias = w.abs()

abs_w, abs_alias


(tensor([3.0, 1.5, 2.0], dtype=Reals, shape=(3,), mutable=True),
 tensor([3.0, 1.5, 2.0], dtype=Reals, shape=(3,), mutable=True))

`absolute()` and `abs()` are equivalent; `abs()` mirrors NumPy's naming.

### 5) Comparisons and boolean masks

Comparisons are the gatekeepers of math and ML: they decide which elements pass a threshold, which errors are large, or which predictions are correct. There are two flavors in PixelPrism:

Python operators like `==` and `<` return a single `bool`. In this implementation `==`, `!=`, `<=`, and `>=` require *all* elements to satisfy the comparison, while `<` and `>` return `True` if *any* element satisfies it.

Tensor methods like `equal()` and `less()` return a **Tensor of booleans**, perfect for masks.

In [17]:
a = pm.Tensor([1.0, 2.0, 3.0])
b = pm.Tensor([1.0, 0.0, 3.0])

op_eq = a == b
op_ne = a != b
op_lt = a < b
op_le = a <= b
op_gt = a > b
op_ge = a >= b

op_eq, op_ne, op_lt, op_le, op_gt, op_ge


(False, True, False, False, True, True)

The operator comparisons return `bool` values (useful for quick checks), while method comparisons return elementwise masks.

In [18]:
mask_eq = a.equal(b)
mask_ne = a.not_equal(b)
mask_lt = a.less(b)
mask_le = a.less_equal(b)
mask_gt = a.greater(b)
mask_ge = a.greater_equal(b)

mask_eq, mask_ne, mask_lt, mask_le, mask_gt, mask_ge


(tensor([True, False, True], dtype=Boolean, shape=(3,), mutable=True),
 tensor([False, True, False], dtype=Boolean, shape=(3,), mutable=True),
 tensor([False, False, False], dtype=Boolean, shape=(3,), mutable=True),
 tensor([True, False, True], dtype=Boolean, shape=(3,), mutable=True),
 tensor([False, True, False], dtype=Boolean, shape=(3,), mutable=True),
 tensor([True, True, True], dtype=Boolean, shape=(3,), mutable=True))

These masks are the building blocks of ML logic: use them to select values, build losses, or filter data.

### 6) Trigonometry and hyperbolic friends

Trigonometry powers rotations, waves, and periodic signals. In ML it pops up in positional encodings, phase features, and Fourier-like representations. PixelPrism follows NumPy: **angles are in radians** by default.

In [19]:
angles = pm.Tensor([0.0, np.pi / 6, np.pi / 2, np.pi])

sin_a = angles.sin()
cos_a = angles.cos()
tan_a = angles.tan()

sin_a, cos_a, tan_a


(tensor([0.0, 0.49999999999999994, 1.0, 1.2246467991473532e-16], dtype=Reals, shape=(4,), mutable=True),
 tensor([1.0, 0.8660254037844387, 6.123233995736766e-17, -1.0], dtype=Reals, shape=(4,), mutable=True),
 tensor([0.0, 0.5773502691896257, 1.633123935319537e+16, -1.2246467991473532e-16], dtype=Reals, shape=(4,), mutable=True))

Inverse trig functions need values in their valid domains (e.g., `arcsin`/`arccos` for inputs in [-1, 1]).

In [20]:
x_unit = pm.Tensor([-1.0, -0.5, 0.0, 0.5, 1.0])

asin_x = x_unit.arcsin()
acos_x = x_unit.arccos()
atan_x = x_unit.arctan()

asin_x, acos_x, atan_x


(tensor([-1.5707963267948966, -0.5235987755982989, 0.0, 0.5235987755982989, 1.5707963267948966], dtype=Reals, shape=(5,), mutable=True),
 tensor([3.141592653589793, 2.0943951023931957, 1.5707963267948966, 1.0471975511965979, 0.0], dtype=Reals, shape=(5,), mutable=True),
 tensor([-0.7853981633974483, -0.4636476090008061, 0.0, 0.4636476090008061, 0.7853981633974483], dtype=Reals, shape=(5,), mutable=True))

`arctan2` is the angle of a 2D vector (y, x). It's great for turning coordinates into directions.

In [21]:
y = pm.Tensor([0.0, 1.0, 1.0])
x = pm.Tensor([1.0, 1.0, -1.0])

angles_2d = y.arctan2(x)
angles_2d


tensor([0.0, 0.7853981633974483, 2.356194490192345], dtype=Reals, shape=(3,), mutable=True)

Hyperbolic functions appear in activation functions (think `tanh`) and in smooth feature transforms.

In [22]:
h = pm.Tensor([-2.0, -1.0, 0.0, 1.0, 2.0])

sinh_h = h.sinh()
cosh_h = h.cosh()
tanh_h = h.tanh()

sinh_h, cosh_h, tanh_h


(tensor([-3.626860407847019, -1.1752011936438014, 0.0, 1.1752011936438014, 3.626860407847019], dtype=Reals, shape=(5,), mutable=True),
 tensor([3.7621956910836314, 1.5430806348152437, 1.0, 1.5430806348152437, 3.7621956910836314], dtype=Reals, shape=(5,), mutable=True),
 tensor([-0.9640275800758169, -0.7615941559557649, 0.0, 0.7615941559557649, 0.9640275800758169], dtype=Reals, shape=(5,), mutable=True))

Inverse hyperbolic functions have their own valid domains; keep inputs in safe ranges when exploring.

In [23]:
h_small = pm.Tensor([-0.9, 0.0, 0.9])

asinh_h = h_small.arcsinh()
atanh_h = h_small.arctanh()

asinh_h, atanh_h


(tensor([-0.8088669356527824, 0.0, 0.8088669356527824], dtype=Reals, shape=(3,), mutable=True),
 tensor([-1.4722194895832204, 0.0, 1.4722194895832204], dtype=Reals, shape=(3,), mutable=True))

In [24]:
h_pos = pm.Tensor([1.0, 2.0, 5.0])

acosh_h = h_pos.arccosh()
acosh_h


tensor([0.0, 1.3169578969248166, 2.2924316695611777], dtype=Reals, shape=(3,), mutable=True)

Need degrees? Convert with `deg2rad()` or `rad2deg()` before or after trig calls.

In [25]:
deg = pm.Tensor([0.0, 30.0, 90.0, 180.0])

rad = deg.deg2rad()
back_to_deg = rad.rad2deg()

rad, back_to_deg


(tensor([0.0, 0.5235987755982988, 1.5707963267948966, 3.141592653589793], dtype=Reals, shape=(4,), mutable=True),
 tensor([0.0, 29.999999999999996, 90.0, 180.0], dtype=Reals, shape=(4,), mutable=True))

## Rendering Tensors with Latex

In [26]:
from pixelprism.math.render.latex import to_latex
to_latex(apbpc)

ImportError: cannot import name 'MathNode' from 'pixelprism.math.math_base' (/home/schaetti/Projets/CODING/PixelPrism/pixelprism/math/math_base.py)

In [ ]:
from pixelprism.math.render import render_latex
latex_image = render_latex(apbpc, output_path="latex_image.svg")
latex_image